# 01. データ検証（Data Validation）

## 目的
- 分析に入る前に、使用するデータの健全性と前提条件を確認する
- 後続のEDA（探索的データ分析）やモデリングに影響する問題  
  （欠損値・データ型・分布の偏り など）を事前に把握する

## 対象範囲
- 対象単位：都道府県（n = 47）
- データ粒度：都道府県単位の集計データ
- 本Notebookでは解釈や考察は行わず、  
  **事実確認のみを目的とする**


In [1]:
import pandas as pd  # データ分析の主役ツール「pandas」をpdという名前で使えるようにする
import numpy as np   # 数値計算が得意な「NumPy」をnpという名前で準備（分析の基本セット）
from pathlib import Path  # ファイルの場所（パス）を直感的に扱うための道具を読み込む

DATA_PATH = Path("../data/processed/nurse_data_clean.csv")  # データの場所を指定（..は「1つ上のフォルダに戻る」という意味）
df = pd.read_csv(DATA_PATH)  # 指定したCSVファイルを読み込んで、表データとしてdfに入れる

pd.set_option("display.max_columns", 100)  # 横に長いデータでも、勝手に省略せず200列まで全部見せる設定
pd.set_option("display.max_rows", 100)     # 縦に長いデータでも、勝手に省略せず200行まで全部見せる設定


In [2]:
# 設定したファイルパスをログに出力して、指定ミスがないか確認
print("file:", DATA_PATH)

# ファイルの実在確認（TrueならOK、Falseならファイルが無い）
print("exists:", DATA_PATH.exists())

# データの構造（列名や値の入り方）を把握するため、先頭5行をプレビュー
df.head()

file: ../data/processed/nurse_data_clean.csv
exists: True


,prefecture,turnover_total,nurse_per_100k,annual_income,night_shift_72h_plus,metro_a,metro_b,job_openings_ratio,turnover_new_grad,turnover_experienced,home_ownership_rate,commute_time,rent_private,hospital_count,large_hospital_count,large_hospital_ratio,hospital_per_100k,population,overtime_hours,night_shift_3_avg,night_shift_2_avg,average_age,population_density
0,北海道,11.5,1306.9,478.90,36.7,0,0,1.12,5.9,16.6,57.0,1.04,53097,534,17,3.18,10.5,5092,5,7.8,4.6,42.0,64.9
1,青森県,8.6,1118.2,435.22,36.5,0,0,1.30,10.7,16.7,71.4,1.01,46924,89,3,3.37,7.5,1184,5,7.7,4.8,42.0,122.8
2,岩手県,6.8,1217.9,458.97,11.8,0,0,1.32,7.8,19.1,70.3,1.03,50136,91,2,2.20,7.8,1163,3,7.5,4.1,46.0,76.1
3,宮城県,9.1,934.4,535.80,30.2,0,0,1.34,7.1,12.4,60.0,1.13,58826,135,7,5.19,6.0,2264,6,8.0,4.7,40.0,310.9
4,秋田県,7.4,1265.3,524.81,25.1,0,0,1.47,5.0,7.3,77.1,1.00,48449,64,3,4.69,7.0,914,4,7.7,4.3,44.9,78.5


In [3]:
# データの規模感（レコード数×カラム数の次元）を把握
print("shape:", df.shape)

# カラム数（特徴量の総数）をカウントして確認
print("columns:", len(df.columns))

# データ型（Dtype）、欠損値（Null）の有無、メモリ使用量を一括でプロファイリング
df.info()

shape: (47, 23)
columns: 23
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   prefecture            47 non-null     object 
 1   turnover_total        47 non-null     float64
 2   nurse_per_100k        47 non-null     float64
 3   annual_income         47 non-null     float64
 4   night_shift_72h_plus  47 non-null     float64
 5   metro_a               47 non-null     int64  
 6   metro_b               47 non-null     int64  
 7   job_openings_ratio    47 non-null     float64
 8   turnover_new_grad     47 non-null     float64
 9   turnover_experienced  47 non-null     float64
 10  home_ownership_rate   47 non-null     float64
 11  commute_time          47 non-null     float64
 12  rent_private          47 non-null     int64  
 13  hospital_count        47 non-null     int64  
 14  large_hospital_count  47 non-null     int64  
 1

In [4]:
# 全カラム名（スキーマ）を、ハンドリングしやすい標準リスト形式で取得
df.columns.tolist()

['prefecture',
 'turnover_total',
 'nurse_per_100k',
 'annual_income',
 'night_shift_72h_plus',
 'metro_a',
 'metro_b',
 'job_openings_ratio',
 'turnover_new_grad',
 'turnover_experienced',
 'home_ownership_rate',
 'commute_time',
 'rent_private',
 'hospital_count',
 'large_hospital_count',
 'large_hospital_ratio',
 'hospital_per_100k',
 'population',
 'overtime_hours',
 'night_shift_3_avg',
 'night_shift_2_avg',
 'average_age',
 'population_density']

In [5]:
# カラム名（スキーマ）の重複を判定し、該当する列名だけを標準リスト形式で抽出
dup_cols = df.columns[df.columns.duplicated()].tolist()

# 重複しているカラム名をログに出力し、データ設計に異常がないかバリデーション
print("duplicated columns:", dup_cols)

duplicated columns: []


In [6]:
# 各カラムの欠損数（Null Count）を集計し、欠損が多い順（降順）にソートして規模を把握
na_count = df.isna().sum().sort_values(ascending=False)

# 全データに対する欠損率（Null Rate）を算出し、データの信頼度を定量的に評価
na_rate = (df.isna().mean() * 100).sort_values(ascending=False)

# 欠損の実数と割合を統合した要約テーブルを作成し、上位30件をレンダリング（プロファイリング表示）
display(pd.DataFrame({"na_count": na_count, "na_rate_%": na_rate}).head(30))

,na_count,na_rate_%
prefecture,0,0.0
rent_private,0,0.0
average_age,0,0.0
night_shift_2_avg,0,0.0
night_shift_3_avg,0,0.0
overtime_hours,0,0.0
population,0,0.0
hospital_per_100k,0,0.0
large_hospital_ratio,0,0.0
large_hospital_count,0,0.0


In [7]:
# 欠損数と欠損率のSeriesを結合し、品質評価用のサマリテーブルを作成
df_na_summary = pd.DataFrame({"na_count": na_count, "na_rate_%": na_rate})

# 欠損のない正常な列を除外し、対応が必要なカラムのみをクエリ抽出して表示
display(df_na_summary.query("na_count > 0"))

,na_count,na_rate_%


In [8]:
# 全カラムの値が完全に一致する行（完全重複レコード）を検知し、その総数をカウント
print("duplicate rows:", df.duplicated().sum())

duplicate rows: 0


In [9]:
# 検証対象とするユニークキー候補（識別子）を変数定義
key = "prefecture"

# 対象キーにおける一意性制約（Uniqueness）違反の総数を検知してログ出力
print("duplicated key:", df[key].duplicated().sum())

# 重複レコードを全件（keep=False）抽出し、内容照合のためにソートして表示
df[df[key].duplicated(keep=False)].sort_values(key)

duplicated key: 0


,prefecture,turnover_total,nurse_per_100k,annual_income,night_shift_72h_plus,metro_a,metro_b,job_openings_ratio,turnover_new_grad,turnover_experienced,home_ownership_rate,commute_time,rent_private,hospital_count,large_hospital_count,large_hospital_ratio,hospital_per_100k,population,overtime_hours,night_shift_3_avg,night_shift_2_avg,average_age,population_density


In [10]:
# 数値型の特徴量（量的変数）のみを抽出し、統計処理用のリストとして定義
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# 数値以外のカテゴリ変数（質的変数）を分離し、エンコーディング等の個別ハンドリング用にリスト化
obj_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

# 特徴量の内訳（次元数）をログ出力し、データの構成比率を把握
print("numeric:", len(num_cols))
print("non-numeric:", len(obj_cols))

# カテゴリ変数のカラム名（スキーマ）を出力し、後続の処理方針を確認
print("non-numeric cols:", obj_cols)

numeric: 22
non-numeric: 1
non-numeric cols: ['prefecture']


In [11]:
# 数値データの基本統計量を算出し、特徴量が多い場所でも視認しやすいよう転置(Transpose)
stats = df[num_cols].describe().T

# 平均値(mean)との乖離(分布の歪み)を比較検証するため、中央値を明示的に列追加
stats["median"] = df[num_cols].median()

# 生成した統計プロファイルをレンダリングし、外れ値や異常検知の一次判断を行う
display(stats)

,count,mean,std,min,25%,50%,75%,max,median
turnover_total,47.0,10.142553,1.887573,6.80,8.700,10.00,11.600,14.20,10.00
nurse_per_100k,47.0,1184.478723,215.356197,744.20,1028.700,1207.00,1340.200,1685.40,1207.00
annual_income,47.0,499.252979,37.632704,416.29,472.180,504.88,530.555,568.09,504.88
night_shift_72h_plus,47.0,35.170213,6.990954,11.80,31.650,36.10,39.000,48.20,36.10
metro_a,47.0,0.170213,0.379883,0.00,0.000,0.00,0.000,1.00,0.00
metro_b,47.0,0.191489,0.397727,0.00,0.000,0.00,0.000,1.00,0.00
job_openings_ratio,47.0,1.395106,0.184083,1.09,1.250,1.41,1.525,1.93,1.41
turnover_new_grad,47.0,7.889362,2.273021,2.80,6.300,7.90,9.400,15.20,7.90
turnover_experienced,47.0,15.119149,3.145434,7.30,13.850,14.70,16.800,22.40,14.70
home_ownership_rate,47.0,66.110638,7.275577,42.60,63.100,67.80,71.000,77.10,67.80


In [12]:
# 数値列を全件スキャンし、論理的に不正な可能性がある「負の値」を含むカラムをリスト化
neg_cols = [c for c in num_cols if (df[c]< 0).any()]

# 異常検知（Anomaly Detection）の結果として、マイナス値を持つカラム名をログ出力
print("has negative values",neg_cols)

has negative values []


In [13]:
# カーディナリティ（ユニーク数）が1以下の「分散0」の特徴量を特定し、情報量のないカラムを抽出
zero_var = [c for c in num_cols if df[c].nunique(dropna=True) <= 1]

# 分析モデルに寄与しない「削除候補（Pruning Candidates）」としてログ出力
print("zero/constant columns:", zero_var)

zero/constant columns: []


In [14]:
def iqr_outliers(s: pd.Series):
    # 欠損値（NaN）は統計計算の邪魔になるため、前処理として除外
    s = s.dropna()

    # データの種類（カーディナリティ）が少なすぎる場合は、統計的な判定が不能なためスキップ
    if s.nunique() < 5:
        return pd.DataFrame()

    # データの分布における第1四分位数（25%点）と第3四分位数（75%点）を算出
    q1, q3 = s.quantile([0.25, 0.75])

    # データの中央50%が含まれる範囲（IQR：四分位範囲）を計算し、ばらつきを定量化
    iqr = q3 - q1

    # 外れ値とみなす下限・上限の閾値（Fence）を定義（一般的な1.5倍ルールを適用）
    lo, hi = q1 - 1.5*iqr, q3 + 1.5*iqr

    # 計算した閾値と、実際のデータの最小・最大値を構造化データとして返却
    return pd.DataFrame({"low": [lo], "high": [hi], "min": [s.min()], "max": [s.max()]})

# 結果を格納する空のリスト（コンテナ）を初期化
out_summary = []

# 全ての数値カラムに対してイテレーション（反復処理）を実行
for c in num_cols:
    # 定義した関数を呼び出し、外れ値判定を実行
    r = iqr_outliers(df[c])

    # 結果が空でない（判定できた）場合のみ処理
    if not r.empty:
        # どのカラムの結果か分かるように、識別用の列を先頭に挿入
        r.insert(0, "col", c)
        out_summary.append(r)

# 結果が存在する場合のみ、リストを結合（Concat）してレポートを表示
if out_summary:
    display(pd.concat(out_summary, ignore_index=True).sort_values("col"))

,col,low,high,min,max
2,annual_income,384.6175,618.1175,416.29,568.09
18,average_age,36.8000,47.6000,38.10,46.00
8,commute_time,0.8525,1.3125,0.56,1.40
7,home_ownership_rate,51.2500,82.8500,42.60,77.10
10,hospital_count,-58.0000,334.0000,43.00,637.00
13,hospital_per_100k,-0.4000,16.0000,3.60,17.70
4,job_openings_ratio,0.8375,1.9375,1.09,1.93
11,large_hospital_count,-5.2500,16.7500,1.00,46.00
12,large_hospital_ratio,-2.1075,10.7125,0.95,10.19
17,night_shift_2_avg,4.0000,5.6000,4.10,5.70


In [15]:
# 分析の「目的変数（ターゲット）」となる、最重要の離職率関連カラムを定義
turnover_cols = ["turnover_total", "turnover_new_grad", "turnover_experienced"]

# 目的変数のいずれかに欠損（NaN）がある行を特定し、分析に使えない「不完全データ」として抽出
# axis=1 は「行単位（横方向）」にチェックするという意味
missing_turnover = df[df[turnover_cols].isna().any(axis=1)]

# 欠損による「データロス（分析対象外となる件数）」をログ出力し、影響の大きさを測定
print("rows with missing turnover:", len(missing_turnover))

# 欠損の発生傾向（特定の都道府県だけで起きているか？など）を目視確認するため、属性を絞ってサンプル表示
display(missing_turnover[["prefecture"] + turnover_cols].head(20))

rows with missing turnover: 0


,prefecture,turnover_total,turnover_new_grad,turnover_experienced


In [16]:
# カテゴリ変数（質的変数）の全カラムに対して、データの中身を確認するイテレーション（反復処理）を開始
for c in obj_cols:
    # 欠損を除去して文字列型にキャスト（型変換）し、実データを先頭5件だけサンプリング（抽出）
    sample = df[c].dropna().astype(str).head(5).tolist()
    
    # データの「フォーマット」や「入力規則」を目視チェック（インスペクション）するため、サンプル値をログ出力
    print(c,"sample",sample)

prefecture sample ['北海道', '青森県', '岩手県', '宮城県', '秋田県']
